# Pre-Entrega2: 
# Generación automatizada de reportes con IA

## Introducción

### Nombre del proyecto:
#### Generación automatizada de reportes con IA

### Presentación del problema a abordar:
En el ámbito de la administración de sistemas y monitoreo IT, herramientas como **Zabbix** generan una gran cantidad de datos sobre disponibilidad, rendimiento y alertas. Aunque estos datos son esenciales, el proceso de convertirlos en reportes significativos para gerentes y stakeholders suele ser tedioso y consume tiempo, ya que requiere:

 - Extraer métricas relevantes de los sistemas.

 - Analizar tendencias y comportamientos en los datos.

 - Presentar resultados de manera clara y profesional.

Este problema es particularmente relevante porque los reportes son fundamentales para tomar decisiones estratégicas, medir acuerdos de nivel de servicio (SLAs) y comunicar resultados de manera efectiva. Sin embargo, los equipos técnicos pueden perder un tiempo valioso en tareas repetitivas en lugar de enfocarse en actividades críticas.


### Desarrollo de la propuesta de solución:
La solución propuesta consiste en desarrollar un sistema automatizado que utilice **modelos de IA** para generar reportes narrativos y también gráficos basados en datos extraídos de Zabbix. Este sistema tendrá dos componentes principales:

**Texto a texto**:

1. Usar prompts que transformen métricas y datos de Zabbix en un análisis narrativo.

2. Por ejemplo, convertir una tabla con tiempos de disponibilidad e indisponibilidad de un cierto servicio en un resumen como:
   - "El servicio X tuvo una disponibilidad del 99.87% esta semana, cumpliendo con el SLA acordado. Hubo una interrupción de 15 minutos el día miércoles, causada por XY razón."


**Texto a imagen**:

1. Generar gráficos y diagramas (disponibilidad semanal, tiempos de indisponibilidad, etc.) basados en las métricas, que complementen el análisis narrativo.

2. Por ejemplo: gráficos de barras para comparar la disponibilidad entre servicios o líneas de tiempo para visualizar tendencias. En caso de no quedar satisfecho con las generaciones de imágenes sobre estos puntos, nuestra contingencia será generar imágenes indicadoras a modo de resumen o pantallazo general de como fue el comportamiento de disponibilidad de los servicios monitoreados.



### Justificación de la viabilidad del proyecto:
Este proyecto es técnicamente viable debido a:

1. Acceso a herramientas: Se utilizará la API de OpenAI para implementar los modelos de IA necesarios.

2. Procesamiento eficiente: Los datos provenientes de Zabbix ya están procesados en un formato amigable para la IA, además, al menos para esta segunda pre-entrega, partimos de una entrada de datos estática en el código.

3. Control de costos: Los prompts se diseñaron para ser breves y optimizados, minimizando por ese lado el gasto de tokens al generar resúmenes y gráficos, no obstante, en algunos escenarios no se acudió al modelo de openai más barato, ya que se requiere de una inteligencia superior para lograr  análisis más complejos.  Así y todo, los costos siguen siendo minimos, ya que la ejecución del script es puntual, en principio es semanal. Además todas las respuestas cuentan con un limitador de tokens para evitar cualquier problema de desborde y topear los costos por respuesta como medida preventiva.


## Objetivos
### Objetivo general:
 - Automatizar la creación de resúmenes narrativos y gráficos representativos a partir de datos de disponibilidad y SLA.

### Objetivos específicos:

 - Generar resúmenes narrativos claros y útiles a partir de datos procesados de Zabbix.
 - Crear gráficos conceptuales que visualicen de forma intuitiva el desempeño semanal.
 - Demostrar la efectividad de los modelos de IA en la generación automatizada de reportes.

## Metodología
**Preparación de los datos:**

 - Extraer los datos de Zabbix usando su API.
 - Procesar y formatear los datos en JSON para su compatibilidad con los modelos de IA.


**Generación del resumen narrativo:**

 - Crear prompts optimizados que alimenten el modelo de texto a texto (GPT).
 - Evaluar y ajustar los resultados generados para garantizar claridad y precisión.

**Creación de gráficos visuales:**

 - Usar un modelo de texto a imagen para generar gráficos conceptuales basados en un score calculado por servicio.

**Pruebas y ajustes:**

 - Validar los resultados con datos reales para garantizar que sean útiles y comprensibles.
 - Optimizar prompts para mejorar la relación costo-beneficio.

## Herramientas y tecnologías

**Librerías:**

 - openai para interacción con modelos de IA.


**Técnicas de prompting:**

 - **Zero-shot prompting:** es eficiente y directo, evitando ejemplos adicionales que podrían aumentar los costos y el procesamiento.
 - **One-shot prompting:** garantiza que el modelo produzca respuestas claras y específicas en situaciones donde el formato esperado es crítico (como el score).



## Implementación



In [78]:
import openai

In [79]:
openai.api_key = "" #Quitamos la key para la segunda pre-entrega

In [80]:
context = """Eres un analista de datos que se encarga de redactar resumenes narrativos detallados para cada servicio, indicando su nivel de disponibilidad, los tiempos de inactividad, las causas de interrupciones en caso
de que las haya, y si cumplieron con el SLA objetivo. 
Además puedes agregar recomendaciones si fuera relevante. El resumen no debe superar en ningún caso los 500 tokens.
"""

In [81]:
data_in = """
{
    "report_week": "2024-11-25 to 2024-12-01",
    "services": [
        {
            "service_name": "Web Server",
            "availability": 99.87,
            "downtime": {
                "total_minutes": 15,
                "events": [
                    {
                        "date": "2024-11-27",
                        "duration_minutes": 15,
                        "cause": "Network failure"
                    }
                ]
            },
            "sla_target": 99.90,
            "alerts": 3,
            "observations": "Met SLA despite minor downtime."
        },
        {
            "service_name": "BackendServer",
            "availability": 100,
            "downtime": {
                "total_minutes": 0,
                "events": [
                ]
            },
            "sla_target": 99.90,
            "alerts": 0,
            "observations": ""
        },
        {
            "service_name": "Database",
            "availability": 99.95,
            "downtime": {
                "total_minutes": 5,
                "events": [
                    {
                        "date": "2024-11-28",
                        "duration_minutes": 5,
                        "cause": "Maintenance"
                    }
                ]
            },
            "sla_target": 99.95,
            "alerts": 1,
            "observations": "Achieved SLA with minimal interruptions."
        }
    ]
}
"""

In [82]:
prompt = f"Estos son los datos a analizar {data_in}"

In [83]:
conversation = [
    {"role": "system", "content": context},
    {"role": "user", "content": prompt}
]

In [84]:
response = openai.ChatCompletion.create(
    model = 'gpt-4',
    messages = conversation,
    max_tokens = 500
)

In [85]:
narrated_summary = response["choices"][0]["message"]["content"].strip()

In [70]:
print(narrated_summary)

Informe Semanal del Servicio: del 25 de noviembre al 1 de diciembre del 2024

1) **Servidor Web**: Con una disponibilidad del 99.87%, este servicio tuvo un tiempo de inactividad de 15 minutos debido a una falla de red registrada el 27 de noviembre del 2024. Aunque estuvo cerca del objetivo del SLA del 99.90%, un ligero desfase existió, desencadenando 3 alertas durante el período. A pesar del tiempo de inactividad menor, logró cumplir con el SLA. 

   Recomendación: Mejorar las infraestructuras de red y establecer protocolos de redundancia para evitar que futuras fallas de red afecten el tiempo de actividad.

2) **BackendServer**: Este servicio mantuvo una disponibilidad perfecta del 100% sin tiempo de inactividad, logrando así superar el objetivo del SLA del 99.90%. No se registraron alertas durante este período.

   Recomendación: Continuar con las prácticas actuales para mantenimiento y monitoreo para asegurar la continuidad del servicio sin interrupciones.

3) **Base de Datos**: Con

In [86]:
score_summary_request = "En base al resumen narrado construído, se requiere calificar el desempeño de los servicios en general en relación al SLA objetivo, y se pide elegir una clasificación de entre 0 y 10, donde 0 es desastroso y 10 es la excelencia. Que número elegirías? Por favor, respondeme solo el número indicador, ni más ni menos."

conversation.append({"role": "user", "content": score_summary_request})

response_2 = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
    messages=conversation,
    max_tokens=10
)

score_summary = response_2['choices'][0]['message']['content'].strip()


In [87]:
print("Score:", score_summary)

Score: 8


In [88]:
score_image = f"Crear un diseño minimalista estilo gauge que refleje un desempeño basado en un score. El diseño es de un color entre rojo (score 0) y verde (score 10), con tonos intermedios según el score. Fondo blanco, diseño limpio y profesional. La imagén resultante no debe de tener números ni letras. El score es {score_summary}"

conversation.append({"role": "user", "content": score_image})

image_response = openai.Image.create(
    prompt=score_image,
    n=1,
    size="256x256"
)

print(image_response['data'][0]['url'])

https://oaidalleapiprodscus.blob.core.windows.net/private/org-zxk0LPtO3ZaaLczq31fRsE41/user-bLzPwqfphC1UZxdNXxq563OB/img-dCwzL6seqJP5cZelRcRs4K0u.png?st=2024-12-04T01%3A30%3A59Z&se=2024-12-04T03%3A30%3A59Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-04T01%3A06%3A49Z&ske=2024-12-05T01%3A06%3A49Z&sks=b&skv=2024-08-04&sig=KKdAj0wEmga70AeiNyhfYxIx/pG33EgtALAMPXrNnj0%3D
